## W2-CRISPR-cBEST workflow


Base-editing with CRISPR-BEST is a DSB-free method that uses a single sgRNA to target a specific genomic location with single-nucleotide–resolution(Figure 3A) which can enable in-vivo protein engineering or as it is often used, for introducing early stop-codons10,12. It can be used to introduce targeted point mutations in the genome of Streptomyces, facilitating studies on gene function and protein engineering. To get started quickly users can download the pCRISPR-cBEST plasmid file, the genome file for S.coelicor (A3) and streptoAIM will generate your plasmids based on the genes you chose to target (Figure 3, S5). 

In [1]:
import sys
import os

# Ensure the src directory is in the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import os
import pandas as pd
from pydna.dseqrecord import Dseqrecord
from datetime import datetime

from streptocad.sequence_loading.sequence_loading import  load_and_process_plasmid, load_and_process_genome_sequences, annotate_dseqrecord, check_and_convert_input, process_specified_gene_sequences_from_record
from streptocad.utils import polymerase_dict,ProjectDirectory, extract_metadata_to_dataframe
from streptocad.crispr.guideRNAcas3_9_12 import extract_sgRNAs, SgRNAargs
from streptocad.cloning.ssDNA_bridging import assemble_plasmids_by_ssDNA_bridging, make_ssDNA_oligos
from streptocad.crispr.crispr_best import identify_base_editing_sites, filter_sgrnas_for_base_editing, process_base_editing
from streptocad.cloning.plasmid_processing import annotate_plasmid_with_sgrnas
from streptocad.primers.primer_generation import find_best_check_primers_from_genome, create_idt_order_dataframe, primers_to_IDT

In [2]:
from pydna.dseqrecord import Dseqrecord

def find_locus_tags(record: Dseqrecord, max_tags: int = None):
    """
    Iterate through a Dseqrecord and return a list of unique locus tags found in its features,
    stopping after `max_tags` tags if specified.

    :param record: pydna.dseqrecord.Dseqrecord object
    :param max_tags: maximum number of locus tags to collect (None for no limit)
    :return: list of locus_tag strings
    """
    tags = []
    seen = set()
    count = 0

    # Ensure the record has features
    if not hasattr(record, 'features'):
        return tags

    for feature in record.features:
        if max_tags is not None and count >= max_tags:
            break

        qualifiers = getattr(feature, 'qualifiers', {})

        for tag in qualifiers.get('locus_tag', []):
            if max_tags is not None and count >= max_tags:
                break
            # only append new tags
            if tag not in seen:
                seen.add(tag)
                tags.append(tag)
                count += 1

        if max_tags is not None and count >= max_tags:
            break

    return tags


## INPUT

In [3]:
# Inputs
# 1 Add genome of choice (genbank, fasta)
path_to_genome = '../../../../data/genomes/Streptomyces_coelicolor_A3_chromosome.gb'
genome = load_and_process_genome_sequences(path_to_genome)[0]

# 2 Add plasmid 
path_to_plasmid = '../../../../data/plasmids/ pCRISPR-cBEST Sequences.gbk'
clean_plasmid = load_and_process_plasmid(path_to_plasmid)

# 3 Choose genes to knock out (list)
locus_tags = find_locus_tags(genome, 100)
genes_to_KO = locus_tags

#### Advanced settings ####
# 3 Choose polymerase and target melting temperature FOR CHECKING PRIMERS
chosen_polymerase = polymerase_dict['Phusion High-Fidelity DNA Polymerase (GC Buffer)']

melting_temperature = 65
primer_concentration = 0.4 
primer_number_increment = 1
flanking_region_number = 500

# 4 Filtering metrics for sgRNAs
gc_upper = 0.99
gc_lower = 0.01
off_target_seed = 13
off_target_upper = 10
cas_type='cas9'
number_of_sgRNAs_per_group = 1
only_stop_codons = True
flanking_region = 500

# 6 Choose overlapping sequences for our plasmid we can use the following
#As per the article **"CRISPR–Cas9, CRISPRi and CRISPR-BEST-mediated genetic manipulation in streptomycetes"** we need the following oligoes: 
#CGGTTGGTAGGATCGACGGC **-N20-** GTTTTAGAGCTAGAAATAGC
up_homology = Dseqrecord('cggttggtaggatcgacggc') # V1 homology : CGGTTGGTAGGATCGACGGC
dw_homology = Dseqrecord('tatgtcctccgagaccggcc') # V1 homology : GTTTTAGAGCTAGAAATAGC

# Computation

In [4]:
from teemi.design.fetch_sequences import read_genbank_files     
sequences = read_genbank_files(path_to_plasmid)
sequences


[SeqRecord(seq=Seq('GGCCGTCGACGACGACGCGTCCGCCTGCCTGCTTTTGGCCGGCGGGCCGGGCCG...CCA'), id='pCRISPR-cBEST(#125689)', name='pCRISPR-cBEST', description='C to T base editor for actinomycetes', dbxrefs=[])]

In [5]:
target_dict, genes_to_KO, annotation_input = check_and_convert_input(genes_to_KO)

print(annotation_input)
if annotation_input == True:
    genome = annotate_dseqrecord(genome, target_dict)



False


In [6]:
len(genes_to_KO)

100

We need to check the input genes so we can find sgRNAs. 

In [7]:

# Initialize SgRNAargs with desired parameters
args = SgRNAargs(genome, 
                genes_to_KO,
                step=['find', 'filter'],
                gc_upper = gc_upper,
                gc_lower = gc_lower,
                off_target_seed = off_target_seed,
                off_target_upper = off_target_upper,
                cas_type='cas9',
                )

sgrna_df = extract_sgRNAs(args)
sgrna_df

sgRNA generated were outside the designated border in SCO0001. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0001. To incorporate this extent borders. Skipping to next locus tag.
Pam was found outside designated locus_tag: SCO0002. To incorporate this extent borders. Skipping to next locus tag.
Pam was found outside designated locus_tag: SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRN

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count
4122,NC_003888.3,SCO0022,22156,-1,-1,270,0.65,CGG,GGCCAGGTTGGGCCAGGTTT,TTGGGCCAGGTTT,-1
4085,NC_003888.3,SCO0022,22156,-1,1,127,0.65,CGG,CACGAGAACCCCGAGCAGTG,ACCCCGAGCAGTG,-1
4096,NC_003888.3,SCO0022,22156,-1,1,275,0.70,TGG,CGCCGAAACCTGGCCCAACC,ACCTGGCCCAACC,-1
4123,NC_003888.3,SCO0022,22156,-1,-1,264,0.75,AGG,GAGTGGGGCCAGGTTGGGCC,GCCAGGTTGGGCC,-1
4124,NC_003888.3,SCO0022,22156,-1,-1,259,0.65,GGG,TCAGCGAGTGGGGCCAGGTT,GTGGGGCCAGGTT,-1
...,...,...,...,...,...,...,...,...,...,...,...
6314,NC_003888.3,SCO0042,33842,1,1,311,0.85,CGG,CGACCCGCTCGCCGACCGGC,CTCGCCGACCGGC,10
10193,NC_003888.3,SCO0069,56225,1,-1,1654,0.80,AGG,GGCTTCGCCGAGCAGGGCGG,CCGAGCAGGGCGG,10
13575,NC_003888.3,SCO0091,77315,1,1,337,0.80,CGG,CGCCACTGGCCGCCCTGGAC,GGCCGCCCTGGAC,10
13409,NC_003888.3,SCO0091,77315,1,-1,185,0.75,CGG,TGCGAGGCTGGTGGTGCGGG,CTGGTGGTGCGGG,10


In [8]:
value_counts = sgrna_df['locus_tag'].value_counts()
value_counts

locus_tag
SCO0069    734
SCO0072    672
SCO0002    564
SCO0033    494
SCO0003    474
          ... 
SCO0074     21
SCO0060     18
SCO0038     14
SCO0045     14
SCO0056     12
Name: count, Length: 94, dtype: int64

In [9]:
len(genome.features)

25824

In [10]:
# Load gene sequences from the genome
gene_sequences = process_specified_gene_sequences_from_record(genome, genes_to_KO)
genes_to_KO_dict = {locus_tag: gene_sequences[locus_tag] for locus_tag in genes_to_KO if locus_tag in gene_sequences}
print(genes_to_KO_dict)
# Identify and annotate base editing sites
sgrna_df_with_editing = identify_base_editing_sites(sgrna_df)

# filter out only sgRNAs that result in base-editing
filtered_sgrna_df_for_base_editing = filter_sgrnas_for_base_editing(sgrna_df_with_editing)
filtered_sgrna_df_for_base_editing

...
{'SCO0001': Dseq(-678)
ATGA..ATAG
TACT..TATC, 'SCO0002': Dseq(-2562)
ATGG..GTAG
TACC..CATC, 'SCO0003': Dseq(-2352)
GTGA..CTGA
CACT..GACT, 'SCO0004': Dseq(-948)
GTGC..GTAA
CACG..CATT, 'SCO0005': Dseq(-2121)
GTGG..GTGA
CACC..CACT, 'SCO0006': Dseq(-1107)
GTGA..GTGA
CACT..CACT, 'SCO0007': Dseq(-447)
GTGT..ATGA
CACA..TACT, 'SCO0008': Dseq(-351)
GTGG..GTAG
CACC..CATC, 'SCO0009': Dseq(-336)
TTGT..GTGA
AACA..CACT, 'SCO0010': Dseq(-1092)
GTGA..GTGA
CACT..CACT, 'SCO0011': Dseq(-810)
GTGC..GTGA
CACG..CACT, 'SCO0012': Dseq(-411)
GTGG..CTGA
CACC..GACT, 'SCO0013': Dseq(-291)
GTGC..GTGA
CACG..CACT, 'SCO0014': Dseq(-279)
ATGG..CTGA
TACC..GACT, 'SCO0015': Dseq(-1233)
ATGT..GTGA
TACA..CACT, 'SCO0016': Dseq(-204)
GTGA..GTGA
CACT..CACT, 'SCO0017': Dseq(-345)
GTGA..CTGA
CACT..GACT, 'SCO0018': Dseq(-867)
ATGG..CTGA
TACC..GACT, 'SCO0019': Dseq(-255)
ATGT..GTAG
TACA..CATC, 'SCO0020': Dseq(-1647)
ATGG..ATGA
TACC..TACT, 'SCO0021': Dseq(-594)
ATGA..GTAG
TACT..CATC, 'SCO0022': Dseq(-423)
CGGC..CTGA
GCCG..GACT

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,editing_context,editable_cytosines
4122,NC_003888.3,SCO0022,22156,-1,-1,270,0.65,CGG,GGCCAGGTTGGGCCAGGTTT,TTGGGCCAGGTTT,-1,0,4
4085,NC_003888.3,SCO0022,22156,-1,1,127,0.65,CGG,CACGAGAACCCCGAGCAGTG,ACCCCGAGCAGTG,-1,"0,0","9,10"
4096,NC_003888.3,SCO0022,22156,-1,1,275,0.70,TGG,CGCCGAAACCTGGCCCAACC,ACCTGGCCCAACC,-1,"0,0,0","4,9,10"
4123,NC_003888.3,SCO0022,22156,-1,-1,264,0.75,AGG,GAGTGGGGCCAGGTTGGGCC,GCCAGGTTGGGCC,-1,"1,0","9,10"
4124,NC_003888.3,SCO0022,22156,-1,-1,259,0.65,GGG,TCAGCGAGTGGGGCCAGGTT,GTGGGGCCAGGTT,-1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6314,NC_003888.3,SCO0042,33842,1,1,311,0.85,CGG,CGACCCGCTCGCCGACCGGC,CTCGCCGACCGGC,10,"0,0,0,1,0","4,5,6,8,10"
10193,NC_003888.3,SCO0069,56225,1,-1,1654,0.80,AGG,GGCTTCGCCGAGCAGGGCGG,CCGAGCAGGGCGG,10,"0,1,0","6,8,9"
13575,NC_003888.3,SCO0091,77315,1,1,337,0.80,CGG,CGCCACTGGCCGCCCTGGAC,GGCCGCCCTGGAC,10,"0,0,1","4,6,10"
13409,NC_003888.3,SCO0091,77315,1,-1,185,0.75,CGG,TGCGAGGCTGGTGGTGCGGG,CTGGTGGTGCGGG,10,1,8


In [11]:
# Process the DataFrame to apply C-to-T mutations
mutated_sgrna_df = process_base_editing(filtered_sgrna_df_for_base_editing, 
                                        genes_to_KO_dict, 
                                        only_stop_codons = False, 
                                        editing_context= False)
mutated_sgrna_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,editing_context,editable_cytosines,mutations
4122,NC_003888.3,SCO0022,22156,-1,-1,270,0.65,CGG,GGCCAGGTTGGGCCAGGTTT,TTGGGCCAGGTTT,-1,0,4,Q92*
4085,NC_003888.3,SCO0022,22156,-1,1,127,0.65,CGG,CACGAGAACCCCGAGCAGTG,ACCCCGAGCAGTG,-1,"0,0","9,10",V47I
4096,NC_003888.3,SCO0022,22156,-1,1,275,0.70,TGG,CGCCGAAACCTGGCCCAACC,ACCTGGCCCAACC,-1,"0,0,0","4,9,10","G96N, G98S"
4123,NC_003888.3,SCO0022,22156,-1,-1,264,0.75,AGG,GAGTGGGGCCAGGTTGGGCC,GCCAGGTTGGGCC,-1,"1,0","9,10",Q92*
9370,NC_003888.3,SCO0066,53141,1,-1,289,0.55,TGG,CCACATCAGGGTGTTCTGGA,AGGGTGTTCTGGA,0,"0,0","4,7",M102I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3404,NC_003888.3,SCO0017,17919,1,1,73,0.90,GGG,CGGCGCCGCCGACGCCGACG,GCCGACGCCGACG,10,"1,1,0,1,0","4,6,7,9,10","A26V, P27L, P28L"
6314,NC_003888.3,SCO0042,33842,1,1,311,0.85,CGG,CGACCCGCTCGCCGACCGGC,CTCGCCGACCGGC,10,"0,0,0,1,0","4,5,6,8,10","P106L, L107F"
10193,NC_003888.3,SCO0069,56225,1,-1,1654,0.80,AGG,GGCTTCGCCGAGCAGGGCGG,CCGAGCAGGGCGG,10,"0,1,0","6,8,9","G556N, E557K"
13575,NC_003888.3,SCO0091,77315,1,1,337,0.80,CGG,CGCCACTGGCCGCCCTGGAC,GGCCGCCCTGGAC,10,"0,0,1","4,6,10","P114L, A116V"


In [12]:
# Process the DataFrame to apply C-to-T mutations
mutated_sgrna_df = process_base_editing(filtered_sgrna_df_for_base_editing, 
                                        genes_to_KO_dict, 
                                        only_stop_codons = True, 
                                        editing_context= False)
mutated_sgrna_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,editing_context,editable_cytosines,mutations
10892,NC_003888.3,SCO0071,60069,-1,-1,0,0.55,AGG,ATGCAGATCAGCTTGGAGGC,TCAGCTTGGAGGC,0,"1,0","4,9",Q2*
14880,NC_003888.3,SCO0105,87907,-1,-1,2,0.70,AGG,GCAGCAGGACGGCACACAGC,GACGGCACACAGC,0,"1,0","5,10",Q3*
7727,NC_003888.3,SCO0058,44253,1,1,9,0.75,CGG,AAGCAGGTGCTGCCCGCGGC,TGCTGCCCGCGGC,1,"1,1","4,10",Q5*
15167,NC_003888.3,SCO0109,90395,-1,-1,8,0.60,CGG,CGATCGACTTCGTCAGCGTC,CTTCGTCAGCGTC,0,"0,0","5,8","R5*, L6F"
7728,NC_003888.3,SCO0058,44253,1,1,5,0.70,CGG,TGGGAAGCAGGTGCTGCCCG,CAGGTGCTGCCCG,1,1,8,Q5*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11315,NC_003888.3,SCO0072,60740,-1,-1,2909,0.80,AGG,GCCCCGAGGCACCACCACCG,GGCACCACCACCG,2,"0,0,1","4,5,10",R972*
11314,NC_003888.3,SCO0072,60740,-1,-1,2937,0.75,CGG,CCCCAGCCGTACCGCTACGC,CGTACCGCTACGC,0,"0,1,0","4,7,8","Q981*, P982L"
11309,NC_003888.3,SCO0072,60740,-1,-1,3027,0.65,CGG,TTCACACAGCCCGACCCCTC,AGCCCGACCCCTC,0,"0,0,1","5,7,10","T1011I, Q1012*, P1013S"
11291,NC_003888.3,SCO0072,60740,-1,-1,3213,0.65,CGG,ATCCAGTACGCCGCTGCGGT,ACGCCGCTGCGGT,0,"0,0","4,9",Q1073*


In [13]:
# Filter the DataFrame to retain only up to 5 sgRNA sequences per locus_tag
filtered_df = mutated_sgrna_df.groupby('locus_tag').head(number_of_sgRNAs_per_group)
filtered_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,editing_context,editable_cytosines,mutations
10892,NC_003888.3,SCO0071,60069,-1,-1,0,0.55,AGG,ATGCAGATCAGCTTGGAGGC,TCAGCTTGGAGGC,0,"1,0","4,9",Q2*
14880,NC_003888.3,SCO0105,87907,-1,-1,2,0.70,AGG,GCAGCAGGACGGCACACAGC,GACGGCACACAGC,0,"1,0","5,10",Q3*
7727,NC_003888.3,SCO0058,44253,1,1,9,0.75,CGG,AAGCAGGTGCTGCCCGCGGC,TGCTGCCCGCGGC,1,"1,1","4,10",Q5*
15167,NC_003888.3,SCO0109,90395,-1,-1,8,0.60,CGG,CGATCGACTTCGTCAGCGTC,CTTCGTCAGCGTC,0,"0,0","5,8","R5*, L6F"
12685,NC_003888.3,SCO0086,72159,-1,-1,3,0.65,CGG,AAGTGGATTCAGCGGGCCGC,TTCAGCGGGCCGC,0,0,10,Q5*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5375,NC_003888.3,SCO0035,29605,-1,-1,284,0.65,TGG,CGAACGACTGGCTGACCCTC,CTGGCTGACCCTC,1,"0,0","5,8",R97*
14308,NC_003888.3,SCO0102,84893,1,1,329,0.75,CGG,CGGCGTGCGAGAACGAGCGG,CGAGAACGAGCGG,0,"1,1","4,8",R113*
5194,NC_003888.3,SCO0033,26527,-1,-1,361,0.80,CGG,GCGCCCTGCAGGTGGTGGCC,GCAGGTGGTGGCC,0,"1,0,0,1","4,5,6,9","A122V, Q124*"
12324,NC_003888.3,SCO0084,70777,1,1,386,0.55,TGG,ACGACAGTTTCTGGAGCTGC,TTTCTGGAGCTGC,1,0,5,Q131*


## Output

In [14]:
# MAke oligoes
list_of_ssDNAs = make_ssDNA_oligos(filtered_df, upstream_ovh = up_homology,
                      downstream_ovh=dw_homology)


# cut plasmid
from Bio.Restriction import NcoI, NheI
linearized_plasmid = sorted(clean_plasmid.cut(NcoI,NheI ), key=lambda x: len(x), reverse=True)[0]

# assemble plasmid
sgRNA_vectors = assemble_plasmids_by_ssDNA_bridging(list_of_ssDNAs,linearized_plasmid)

In [15]:
# Constructing a meaningful name, ID, and description for the assembled plasmid using user input
targeting_info = []
for index, row in filtered_df.iterrows():
    formatted_str = f"pCRISPR-BEST_{row['locus_tag']}_p{row['sgrna_loc']}"
    targeting_info.append(formatted_str)

for i in range(len(sgRNA_vectors)):
    sgRNA_vectors[i].name = f'{targeting_info[i]}_#{i+1}'
    sgRNA_vectors[i].id = sgRNA_vectors[i].name  # Using the same value for ID as for name for simplicity
    sgRNA_vectors[i].description = f'Assembled plasmid targeting {", ".join(genes_to_KO)} for base-editing, assembled using StreptoCAD.'

# annotate plasmids
for plasmid in sgRNA_vectors: 
    annotate_plasmid_with_sgrnas(plasmid, filtered_df)

# PRINT
print_plasmids = False
if print_plasmids: 
    for vector in sgRNA_vectors: 
        vector.write(f"../../data/plasmids/sgRNA_plasmids_pCRISPR_cBEST/{vector.id}.gb")

In [16]:
integration_names = filtered_df.apply(lambda row: f"sgRNA_{row['locus_tag']}({row['sgrna_loc']})", axis=1).tolist()
plasmid_metadata_df = extract_metadata_to_dataframe(sgRNA_vectors,
                                                    clean_plasmid,
                                                    integration_names)

plasmid_metadata_df

,plasmid_name,date,original_plasmid,integration,size
0,pCRISPR-BEST_SCO0071_p0_#1,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0071(0),11817
1,pCRISPR-BEST_SCO0105_p2_#2,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0105(2),11817
2,pCRISPR-BEST_SCO0058_p9_#3,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0058(9),11817
3,pCRISPR-BEST_SCO0109_p8_#4,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0109(8),11817
4,pCRISPR-BEST_SCO0086_p3_#5,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0086(3),11817
...,...,...,...,...,...
84,pCRISPR-BEST_SCO0035_p284_#85,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0035(284),11817
85,pCRISPR-BEST_SCO0102_p329_#86,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0102(329),11817
86,pCRISPR-BEST_SCO0033_p361_#87,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0033(361),11817
87,pCRISPR-BEST_SCO0084_p386_#88,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0084(386),11817


# IDT primers

In [17]:
full_idt = primers_to_IDT(list_of_ssDNAs)
full_idt

,Name,Sequence,Concentration,Purification
0,SCO0071_loc_0,cggttggtaggatcgacggcATGCAGATCAGCTTGGAGGCtatgtc...,25nm,STD
1,SCO0105_loc_2,cggttggtaggatcgacggcGCAGCAGGACGGCACACAGCtatgtc...,25nm,STD
2,SCO0058_loc_9,cggttggtaggatcgacggcAAGCAGGTGCTGCCCGCGGCtatgtc...,25nm,STD
3,SCO0109_loc_8,cggttggtaggatcgacggcCGATCGACTTCGTCAGCGTCtatgtc...,25nm,STD
4,SCO0086_loc_3,cggttggtaggatcgacggcAAGTGGATTCAGCGGGCCGCtatgtc...,25nm,STD
...,...,...,...,...
84,SCO0035_loc_284,cggttggtaggatcgacggcCGAACGACTGGCTGACCCTCtatgtc...,25nm,STD
85,SCO0102_loc_329,cggttggtaggatcgacggcCGGCGTGCGAGAACGAGCGGtatgtc...,25nm,STD
86,SCO0033_loc_361,cggttggtaggatcgacggcGCGCCCTGCAGGTGGTGGCCtatgtc...,25nm,STD
87,SCO0084_loc_386,cggttggtaggatcgacggcACGACAGTTTCTGGAGCTGCtatgtc...,25nm,STD


## Folder with all the generated I/O

In [18]:
input_files = [
    {"name": "input_genome.gb", "content": genome},
    {"name": "input_plasmid.gb", "content": clean_plasmid}
]

output_files = [
    {"name": "cBEST_w_sgRNAs.gb", "content": sgRNA_vectors}, # LIST OF Dseqrecords
    {"name": "full_idt.csv", "content": full_idt},
    {"name": "mutated_sgrna_df.csv", "content": mutated_sgrna_df},
    {"name": "filtered_df.csv", "content": filtered_df},
    {"name": "plasmid_metadata_df.csv", "content": plasmid_metadata_df},

]

input_values = {
    "genes_to_knockout": genes_to_KO,
    "polymerase_settings": {
        "chosen_polymerase": chosen_polymerase,
        "melting_temperature": melting_temperature,
        "primer_concentration": primer_concentration,
        "primer_number_increment": primer_number_increment,
        "flanking_region_number": flanking_region_number
    },
    "filtering_metrics": {
        "gc_upper": gc_upper,
        "gc_lower": gc_lower,
        "off_target_seed": off_target_seed,
        "off_target_upper": off_target_upper,
        "cas_type": cas_type,
        "number_of_sgRNAs_per_group": number_of_sgRNAs_per_group
    },
    "overlapping_sequences": {
        "up_homology": str(up_homology),
        "dw_homology": str(dw_homology)
    }
}


# Paths to Markdown files
markdown_file_paths = [
    "../../protocols/conjugation_protcol.md",
    "../../protocols/single_target_crispr_plasmid_protcol.md"

]

# Data and time
timestamp = datetime.utcnow().isoformat()

# Create project directory structure
project_directory = ProjectDirectory(
    project_name=f"CRISPR_cBEST_workflow_{timestamp}",
    input_files=input_files,
    output_files=output_files,
    input_values=input_values,
    markdown_file_paths=markdown_file_paths
)


# DO You want to save the folder? 
save_zip_folder = False 

if save_zip_folder: 
    # Generate the project directory structure and get the zip content
    zip_content = project_directory.create_directory_structure(create_directories=False)

    # Save the zip file to disk (optional)
    with open("project_structure.zip", "wb") as f:
        f.write(zip_content)